In [14]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, TrainingArguments, Trainer
from tqdm import tqdm
import re
from sklearn.metrics import precision_recall_fscore_support
import json
import random
from datasets import Dataset


In [15]:
df = pd.read_csv("../datasets/latest/merged_output_combined_second_iter.csv")

In [16]:
df

,raw_text,extracted_role,clean_text,skills,confidence,GT,processed
0,C:\Workspace\java\scrape_indeed\dba_part_1\1.h...,Database Administrator,database administrator database administrator ...,NaN,0.0,0,False
1,C:\Workspace\java\scrape_indeed\dba_part_1\10....,Database Administrator,database administrator sql microsoft powerpoin...,NaN,0.0,0,False
2,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,oracle database administrator oracle database ...,NaN,0.0,0,False
3,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,amazon redshift administrator and etl develope...,NaN,0.0,0,False
4,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,scrum master oracle database administrator scr...,NaN,0.0,0,False
...,...,...,...,...,...,...,...
70321,Description\n\nAssisting with data and indirec...,program manager,description assisting with data and indirect p...,NaN,0.0,0,False
70322,Senior Business Intelligence DeveloperLos Ange...,business intelligence developer,senior business intelligence developerlos ange...,NaN,0.0,0,False
70323,Title: Project ManagerDuration: 4 Months – Lon...,project manager,title project managerduration 4 months long te...,NaN,0.0,0,False
70324,About The Position\n\nThe Research Scientist w...,research scientist i,about the position the research scientist will...,NaN,0.0,0,False


In [17]:
df[df['skills'].notna()]

,raw_text,extracted_role,clean_text,skills,confidence,GT,processed
3562,C:\Workspace\java\scrape_indeed\dba_part_1\103...,Database Administrator,database and sysadmin consultant database admi...,"database administration, SQL, Oracle, Oracle 1...",0.95,1,True
3617,C:\Workspace\java\scrape_indeed\dba_part_1\108...,Database Administrator,f 35 suitability analyst f 35 database adminis...,"database administration, project leadership, s...",0.95,1,True
3705,C:\Workspace\java\scrape_indeed\dba_part_1\116...,Database Administrator,application developer blackboard administrator...,"javascript, objective c, php, html5, jquery, h...",0.95,1,True
3716,C:\Workspace\java\scrape_indeed\dba_part_1\117...,Database Administrator,business intelligence developer database analy...,"SQL Server Management Studio, Microsoft Busine...",0.95,1,True
3724,C:\Workspace\java\scrape_indeed\dba_part_1\118...,Database Administrator,oracle sql database administrator oracle dba o...,"Oracle SQL, Database Administration, Data Mode...",0.95,1,True
...,...,...,...,...,...,...,...
70179,Job Title\n\nProgram Specialist I\n\nAgency\n\...,program specialist i,job title program specialist i agency texas a ...,"general office support, correspondence prepara...",0.95,1,True
70193,"About Mortenson\n\nAs a Top 25 builder, develo...",engineering project manager i,about mortenson as a top 25 builder developer ...,"project management, scope management, schedule...",0.95,1,True
70245,Job Description\n\nWhat if you could use your ...,"software engineer, backend",job description what if you could use your tec...,"object-oriented programming, test-driven devel...",0.95,1,True
70271,Are you ready to take your career to new heigh...,embedded systems engineer,are you ready to take your career to new heigh...,"embedded software design, C programming, Micro...",0.95,1,True


In [18]:
# Load the datasets

job_postings_df = pd.read_csv('../datasets/job_postings/computing_desc_job_posting.csv')  # Adjust filename as needed



print("\nJob postings dataset structure:")
print(job_postings_df.head())
print(job_postings_df.columns)


Job postings dataset structure:
      job_id        company_name                        title  \
0   11009123   PGAV Destinations            project architect   
1   69333422     Staffing Theory    product marketing manager   
2  133130219                 NaN            software engineer   
3  175485704                GOYT            software engineer   
4  266825034  Recruitment Design  software support specialist   

                                         description  max_salary pay_period  \
0  PGAV Destinations is seeking a self-motivated ...         NaN        NaN   
1  A leading pharmaceutical company committed to ...         NaN        NaN   
2  Education Bachelor's degree in software, math,...         NaN        NaN   
3  Job Description:GOYT is seeking a skilled and ...         NaN        NaN   
4  Are you driven by the thrill of solving proble...     65000.0     YEARLY   

                        location  company_id  views  med_salary  ...  \
0                   St Louis,

In [19]:
job_postings_df.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'pay_period', 'location', 'company_id', 'views', 'med_salary',
       'min_salary', 'formatted_work_type', 'applies', 'original_listed_time',
       'remote_allowed', 'job_posting_url', 'application_url',
       'application_type', 'expiry', 'closed_time',
       'formatted_experience_level', 'skills_desc', 'listed_time',
       'posting_domain', 'sponsored', 'work_type', 'currency',
       'compensation_type', 'normalized_salary', 'zip_code', 'fips',
       'matched_category', 'standardized_salary', 'description_length',
       'cleaned_description'],
      dtype='object')

In [20]:
# Load the datasets

resume_df = pd.read_csv('combined_resume_data.csv')  # Adjust filename as needed



print("\nResume dataset structure:")
print(resume_df.head())
print(resume_df.columns)


Resume dataset structure:
                                            raw_text                    role  \
0  C:\Workspace\java\scrape_indeed\dba_part_1\1.h...  Database Administrator   
1  C:\Workspace\java\scrape_indeed\dba_part_1\10....  Database Administrator   
2  C:\Workspace\java\scrape_indeed\dba_part_1\100...  Database Administrator   
3  C:\Workspace\java\scrape_indeed\dba_part_1\100...  Database Administrator   
4  C:\Workspace\java\scrape_indeed\dba_part_1\100...  Database Administrator   

                                          clean_text  \
0  database administrator database administrator ...   
1  database administrator sql microsoft powerpoin...   
2  oracle database administrator oracle database ...   
3  amazon redshift administrator and etl develope...   
4  scrum master oracle database administrator scr...   

                  source_file  
0  Database_Administrator.csv  
1  Database_Administrator.csv  
2  Database_Administrator.csv  
3  Database_Administrator.c

In [21]:
# Add resume_id as primary key
resume_df.reset_index(inplace=True)
resume_df.rename(columns={'index': 'resume_id'}, inplace=True)

In [22]:
resume_df

,resume_id,raw_text,role,clean_text,source_file
0,0,C:\Workspace\java\scrape_indeed\dba_part_1\1.h...,Database Administrator,database administrator database administrator ...,Database_Administrator.csv
1,1,C:\Workspace\java\scrape_indeed\dba_part_1\10....,Database Administrator,database administrator sql microsoft powerpoin...,Database_Administrator.csv
2,2,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,oracle database administrator oracle database ...,Database_Administrator.csv
3,3,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,amazon redshift administrator and etl develope...,Database_Administrator.csv
4,4,C:\Workspace\java\scrape_indeed\dba_part_1\100...,Database Administrator,scrum master oracle database administrator scr...,Database_Administrator.csv
...,...,...,...,...,...
32246,32246,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer,servicenow developer servicenow developer serv...,Web_Developer.csv
32247,32247,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer,android developer android developer route plan...,Web_Developer.csv
32248,32248,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer,ui developer ui web developer ui developer ui ...,Web_Developer.csv
32249,32249,C:\Workspace\java\scrape_indeed\web_dev_part_1...,Web Developer,web developer javascript php front end wordpre...,Web_Developer.csv


In [23]:
ENTITY_TYPES = [
    "PL",      # Programming Language
    "FW",      # Framework
    "DB",      # Database
    "CP",      # Cloud Platform
    "DO",      # DevOps
    "NS",      # Network & Security
    "DAS",     # Data Analysis & Science
    "SWE",     # Software Engineering
    "PM",      # Project Management
    "EC",      # Education Certification
    "SS"       # Soft Skills
]

# Load pre-trained BERT model for NER
# Several options for NER models:
# 1. dslim/bert-base-NER - General NER model (person, org, location, etc.)
# 2. jean-baptiste/roberta-large-ner-english - More accurate general NER
# 3. tomaarsen/span-marker-mbert-base-multinerd - Good for specialized entities
# 4. beki/distilbert-base-uncased-finetuned-skillner - Specifically for skills

# Let's use a more powerful model for general NER as starting point
model_name = "jean-baptiste/roberta-large-ner-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Note: For production use, fine-tuning on computing job data would be ideal

# Create NER pipeline
ner = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")


In [ ]:

def preprocess_text(text):
    """Clean and prepare text for NER"""
    if not isinstance(text, str):
        return ""
    
    # Remove extra whitespace and normalize text
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def extract_entities(text, ner_pipeline):
    """Extract entities from text using BERT NER and keyword matching"""
    # Define keyword lists for each entity type
    keyword_mappings = {
        "PL": ["python", "java", "javascript", "c++", "c#", "go", "rust", "php", "ruby", "swift", 
               "typescript", "kotlin", "scala", "perl", "bash", "powershell", "r", "matlab", "dart", "lua"],
        
        "FW": ["react", "angular", "vue", "django", "flask", "spring", "laravel", "express", "symfony", 
               "bootstrap", "jquery", "rails", "asp.net", "node.js", "next.js", "flutter", "xamarin", "qt"],
        
        "DB": ["sql", "mysql", "postgresql", "mongodb", "oracle", "cassandra", "redis", "dynamodb", 
               "sqlite", "mariadb", "couchdb", "neo4j", "firebase", "elasticsearch", "snowflake", "bigtable"],
        
        "CP": ["aws", "azure", "gcp", "google cloud", "heroku", "digitalocean", "alibaba", "ibm cloud", 
               "oracle cloud", "salesforce", "s3", "ec2", "lambda", "kubernetes", "eks", "gke", "aks"],
        
        "DO": ["docker", "kubernetes", "jenkins", "gitlab", "github actions", "circleci", "terraform", 
               "ansible", "puppet", "chef", "prometheus", "grafana", "elk", "ci/cd", "devops", "sre"],
        
        "NS": ["firewall", "vpn", "encryption", "ssl", "tls", "authentication", "authorization", 
               "oauth", "jwt", "kerberos", "ldap", "penetration testing", "security", "cybersecurity"],
        
        "DAS": ["machine learning", "data mining", "pandas", "numpy", "tensorflow", "pytorch", "big data", 
                "hadoop", "spark", "tableau", "power bi", "data science", "statistics", "ai", "nlp"],
        
        "SWE": ["agile", "scrum", "object-oriented", "oop", "design patterns", "architecture", 
                "microservices", "rest api", "soap", "uml", "solid", "tdd", "bdd", "clean code"],
        
        "PM": ["jira", "confluence", "trello", "asana", "scrum master", "product owner", "kanban", 
               "project management", "pmp", "agile", "waterfall", "lean", "sprint planning"],
        
        "EC": ["bachelor", "master", "phd", "certificate", "certification", "aws certified", "cisco", 
               "comptia", "microsoft certified", "google certified", "pmp", "itil", "cisa", "cissp"],
        
        "SS": ["teamwork", "communication", "leadership", "problem-solving", "creativity", "collaboration", 
               "time management", "adaptability", "critical thinking", "interpersonal", "presentation"]
    }
    
    # Initialize results dictionary with empty lists for all entity types
    organized_entities = {entity_type: [] for entity_type in ENTITY_TYPES}
    
    # Handle empty or invalid input
    if not isinstance(text, str) or not text.strip():
        return organized_entities
    
    # Preprocess text
    processed_text = preprocess_text(text)
    if not processed_text:
        return organized_entities
    
    try:
        # Apply keyword-based extraction (simpler and more reliable for technical terms)
        lower_text = processed_text.lower()
        for category, keywords in keyword_mappings.items():
            for keyword in keywords:
                # Use word boundary check to avoid partial matches
                pattern = r'\b' + re.escape(keyword) + r'\b'
                if re.search(pattern, lower_text):
                    organized_entities[category].append(keyword)
        
        # Apply NER model for entities that might be missed by keywords
        max_length = 512  # BERT token limit
        words = processed_text.split()
        
        # Process text in chunks if needed
        if len(words) > max_length - 10:
            chunks = []
            for i in range(0, len(words), max_length - 10):
                chunks.append(" ".join(words[i:i+max_length-10]))
        else:
            chunks = [processed_text]
        
        # Process each chunk
        for chunk in chunks:
            try:
                # Get NER results
                ner_results = ner_pipeline(chunk)
                
                # Process each entity
                for entity in ner_results:
                    # Extract the entity word and group
                    word = entity.get("word", "").strip()
                    group = entity.get("entity_group", "")
                    
                    # Skip empty words
                    if not word:
                        continue
                    
                    # Map entity groups to our categories
                    if group == "MISC":
                        # For MISC entities, check which technical category they might belong to
                        word_lower = word.lower()
                        for cat, kw_list in keyword_mappings.items():
                            if any(kw in word_lower for kw in kw_list):
                                organized_entities[cat].append(word)
                                break
                        else:
                            # If no match found, default to Programming Language
                            organized_entities["PL"].append(word)
                    
                    elif group == "ORG":
                        # Organizations could be companies or cloud platforms
                        word_lower = word.lower()
                        if any(kw in word_lower for kw in keyword_mappings["CP"]):
                            organized_entities["CP"].append(word)
                    
                    elif group == "PER" and "years" in chunk.lower()[max(0, entity.get("start", 0)-20):entity.get("end", 0)+20]:
                        # People names near "years" might indicate experience
                        organized_entities["SS"].append(word)
            
            except Exception as e:
                print(f"Error in NER processing: {e}")
                # Continue with other chunks
        
        # Remove duplicates (case-insensitive)
        for entity_type in ENTITY_TYPES:
            seen = set()
            unique_list = []
            for item in organized_entities[entity_type]:
                item_lower = item.lower()
                if item_lower not in seen:
                    seen.add(item_lower)
                    unique_list.append(item)
            organized_entities[entity_type] = unique_list
            
    except Exception as e:
        print(f"Error in extract_entities: {e}")
        import traceback
        traceback.print_exc()
    
    return organized_entities

In [ ]:
# Simple test to see NER model output structure
test_text = "Python developer with experience in AWS and SQL"
result = ner(test_text)
print("NER output structure:", result)

NER output structure: [{'entity_group': 'MISC', 'score': 0.94707745, 'word': ' Python', 'start': 0, 'end': 6}, {'entity_group': 'MISC', 'score': 0.52592003, 'word': ' AWS', 'start': 36, 'end': 39}, {'entity_group': 'MISC', 'score': 0.95191735, 'word': ' SQL', 'start': 44, 'end': 47}]


In [ ]:
# For demonstration, process a small sample
sample_size = 5  # Adjust as needed

print("\nProcessing sample resumes...")
for idx, row in tqdm(resume_df.head(sample_size).iterrows(), total=sample_size):
    processed_text = preprocess_text(row['raw_text'])
    if processed_text:
        entities = extract_entities(processed_text, ner)
        print(f"\nResume ID: {row['resume_id']}")
        for entity_type, values in entities.items():
            print(f"  {entity_type}: {', '.join(values[:5])}{'...' if len(values) > 5 else ''}")

print("\nProcessing sample job postings...")
for idx, row in tqdm(job_postings_df.head(sample_size).iterrows(), total=sample_size):
    processed_text = preprocess_text(row['description'])
    if processed_text:
        entities = extract_entities(processed_text, ner)
        print(f"\nJob ID: {row['job_id']}")
        for entity_type, values in entities.items():
            print(f"  {entity_type}: {', '.join(values[:5])}{'...' if len(values) > 5 else ''}")


Processing sample resumes...


 20%|██        | 1/5 [00:04<00:18,  4.52s/it]


Resume ID: 0
  PL: java, Production SQL Server, SQL Server, SSIS, OLTP...
  FW: 
  DB: sql, T-SQL
  CP: 
  DO: 
  NS: security
  DAS: Always On Availability
  SWE: 
  PM: 
  EC: bachelor
  SS: communication


 40%|████      | 2/5 [00:06<00:08,  2.83s/it]


Resume ID: 1
  PL: java, Microsoft PowerPoint, Windows XP, Microsoft office, Assembly Language...
  FW: 
  DB: sql
  CP: 
  DO: 
  NS: security
  DAS: 
  SWE: 
  PM: 
  EC: 
  SS: 


...


Job ID: 133130219
  PL: 
  FW: 
  DB: 
  CP: 
  DO: 
  NS: 
  DAS: 
  SWE: 
  PM: 
  EC: bachelor
  SS: 


 80%|████████  | 4/5 [00:03<00:00,  1.11it/s]


Job ID: 175485704
  PL: javascript, php, HTML, CSS
  FW: 
  DB: mysql
  CP: 
  DO: 
  NS: 
  DAS: 
  SWE: object-oriented
  PM: 
  EC: 
  SS: communication, collaboration


100%|██████████| 5/5 [00:04<00:00,  1.07it/s]


Job ID: 266825034
  PL: javascript, php, Web Application, Web Development, Spanish/Portuguese
  FW: 
  DB: 
  CP: 
  DO: 
  NS: 
  DAS: 
  SWE: 
  PM: 
  EC: bachelor
  SS: communication


In [ ]:
gt =  pd.read_csv("./final_merged.csv")

In [ ]:
gt.columns

Index(['raw_text', 'extracted_role', 'clean_text', 'skills', 'confidence',
       'GT', 'processed'],
      dtype='object')

In [ ]:
row = gt.loc[34440]
row['skills']

'Oracle Database Administration, SQL, RMAN, Linux, Database Installation and Configuration, Database Upgrades and Patching, Database Migration, Performance Tuning, Database Refreshes, Security Management, Backup and Recovery, Database Optimization, Data Migration, Oracle Virtual Server, Oracle VMs, Golden Gate, Oracle Data Guard, Oracle Enterprise Manager (OEM), TOAD, Oracle Grid Control, ASM, RAC, PL/SQL, Unix Shell Scripting, ETL, Data Warehousing, Partitioning, Advanced Compression, Transparent Data Encryption (TDE), Oracle Advanced Security, SQL Loader, Data Pump, Flashback Technology, Log Miner, Automatic Database Diagnostic Monitor (ADDM), AWR Analysis, ASH Monitoring, Oracle 12c Multitenant, Container and Pluggable Databases, Storage Area Networks (SAN), Network Attached Storage (NAS), Exadata, Veritas NetBackup, IBM Tivoli Storage Manager (TSM), Crontab, Communication Skills, Documentation Skills, Problem-Solving, Team Collaboration, Time Management, Attention to Detail, Adapta

In [ ]:
def evaluate_extraction(extracted_entities, ground_truth_skills):
    """
    Evaluate the extraction against ground truth skills
    
    Args:
        extracted_entities: Dictionary with entity types as keys and lists of entities as values
        ground_truth_skills: Comma-separated string of skills from ground truth
    
    Returns:
        Dictionary with precision, recall, and F1 score
    """
    # Flatten all extracted entities into a single list
    all_extracted = []
    for entity_list in extracted_entities.values():
        all_extracted.extend([e.lower() for e in entity_list])
    
    # Process ground truth skills
    if isinstance(ground_truth_skills, str):
        ground_truth = [s.strip().lower() for s in ground_truth_skills.split(',')]
    else:
        ground_truth = []
    
    # Calculate metrics
    true_positives = sum(1 for skill in all_extracted if skill in ground_truth)
    false_positives = sum(1 for skill in all_extracted if skill not in ground_truth)
    false_negatives = sum(1 for skill in ground_truth if skill not in all_extracted)
    
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {0
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "true_positives": true_positives,
        "false_positives": false_positives,
        "false_negatives": false_negatives
    }


In [ ]:
# Filter to only use rows marked as ground truth (GT = 1)
ground_truth_df = gt[gt['GT'] == 1]

# Evaluate on the ground truth data
results = []
for idx, row in ground_truth_df.iterrows():
    # Extract entities
    entities = extract_entities(row['raw_text'], ner)
    
    # Evaluate against ground truth skills
    metrics = evaluate_extraction(entities, row['skills'])
    
    results.append({
        'id': idx,
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'f1': metrics['f1']
    })
    
    print(f"\nDocument ID: {idx}")
    print(f"Precision: {metrics['precision']:.2f}")
    print(f"Recall: {metrics['recall']:.2f}")
    print(f"F1 Score: {metrics['f1']:.2f}")

# Calculate average metrics
avg_precision = sum(r['precision'] for r in results) / len(results)
avg_recall = sum(r['recall'] for r in results) / len(results)
avg_f1 = sum(r['f1'] for r in results) / len(results)

print(f"\nOverall Results:")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")


Document ID: 45
Precision: 0.28
Recall: 0.12
F1 Score: 0.17

Document ID: 100
Precision: 0.20
Recall: 0.10
F1 Score: 0.13

Document ID: 188
Precision: 0.68
Recall: 0.38
F1 Score: 0.49

Document ID: 199
Precision: 0.35
Recall: 0.19
F1 Score: 0.24

Document ID: 207
Precision: 0.29
Recall: 0.25
F1 Score: 0.27

Document ID: 293
Precision: 0.67
Recall: 0.06
F1 Score: 0.12

Document ID: 450
Precision: 0.20
Recall: 0.03
F1 Score: 0.06

Document ID: 462
Precision: 0.34
Recall: 0.19
F1 Score: 0.24

Document ID: 557
Precision: 0.44
Recall: 0.23
F1 Score: 0.30

Document ID: 621
Precision: 0.38
Recall: 0.27
F1 Score: 0.32

Document ID: 663
Precision: 0.12
Recall: 0.16
F1 Score: 0.14

Document ID: 671
Precision: 0.36
Recall: 0.20
F1 Score: 0.26

Document ID: 797
Precision: 0.11
Recall: 0.06
F1 Score: 0.08

Document ID: 889
Precision: 0.07
Recall: 0.17
F1 Score: 0.10

Document ID: 949
Precision: 0.00
Recall: 0.00
F1 Score: 0.00

Document ID: 1009
Precision: 0.19
Recall: 0.07
F1 Score: 0.10

Documen

In [ ]:
# Filter to only use rows marked as ground truth (GT = 1)
ground_truth_df = gt[gt['GT'] == 1]

In [ ]:
def determine_entity_type(skill):
    """
    Maps a given skill to one of the predefined entity types.
    Returns "MISC" if no matching category is found.
    
    Parameters:
    skill (str): The skill to categorize
    
    Returns:
    str: Entity type code (PL, FW, DB, etc.) or "MISC" for unmatched skills
    """
    if not skill:
        return "MISC"
        
    skill_lower = skill.lower()
    
    keyword_mappings = {
        "PL": ["python", "java", "javascript", "c++", "c#", "go", "rust", "php", "ruby", "swift",
               "typescript", "kotlin", "scala", "perl", "bash", "powershell", "r", "matlab", "dart", "lua"],
        
        "FW": ["react", "angular", "vue", "django", "flask", "spring", "laravel", "express", "symfony",
               "bootstrap", "jquery", "rails", "asp.net", "node.js", "next.js", "flutter", "xamarin", "qt"],
        
        "DB": ["sql", "mysql", "postgresql", "mongodb", "oracle", "cassandra", "redis", "dynamodb",
               "sqlite", "mariadb", "couchdb", "neo4j", "firebase", "elasticsearch", "snowflake", "bigtable"],
        
        "CP": ["aws", "azure", "gcp", "google cloud", "heroku", "digitalocean", "alibaba", "ibm cloud",
               "oracle cloud", "salesforce", "s3", "ec2", "lambda", "kubernetes", "eks", "gke", "aks"],
        
        "DO": ["docker", "kubernetes", "jenkins", "gitlab", "github actions", "circleci", "terraform",
               "ansible", "puppet", "chef", "prometheus", "grafana", "elk", "ci/cd", "devops", "sre"],
        
        "NS": ["firewall", "vpn", "encryption", "ssl", "tls", "authentication", "authorization",
               "oauth", "jwt", "kerberos", "ldap", "penetration testing", "security", "cybersecurity"],
        
        "DAS": ["machine learning", "data mining", "pandas", "numpy", "tensorflow", "pytorch", "big data",
                "hadoop", "spark", "tableau", "power bi", "data science", "statistics", "ai", "nlp"],
        
        "SWE": ["agile", "scrum", "object-oriented", "oop", "design patterns", "architecture",
                "microservices", "rest api", "soap", "uml", "solid", "tdd", "bdd", "clean code"],
        
        "PM": ["jira", "confluence", "trello", "asana", "scrum master", "product owner", "kanban",
               "project management", "pmp", "agile", "waterfall", "lean", "sprint planning"],
        
        "EC": ["bachelor", "master", "phd", "certificate", "certification", "aws certified", "cisco",
               "comptia", "microsoft certified", "google certified", "pmp", "itil", "cisa", "cissp"],
        
        "SS": ["teamwork", "communication", "leadership", "problem-solving", "creativity", "collaboration",
               "time management", "adaptability", "critical thinking", "interpersonal", "presentation"]
    }
    
    # Check each category's keywords for matches
    for category, keywords in keyword_mappings.items():
        for keyword in keywords:
            # Check if the keyword is in the skill text (as a whole word or part of a compound word)
            if keyword in skill_lower:
                return category
    
    # If no match is found
    return "MISC"

In [ ]:
# Create NER training data in the correct format
training_data = []

for idx, row in ground_truth_df.iterrows():
    text = row['raw_text']
    skills_str = row['skills']
    
    # Skip if text or skills are missing
    if not isinstance(text, str) or not isinstance(skills_str, str):
        continue
    
    # Parse skills
    skills = [skill.strip() for skill in skills_str.split(',')]
    
    # Create annotations
    annotations = []
    for skill in skills:
        # Find all occurrences of each skill in the text
        for match in re.finditer(r'\b' + re.escape(skill) + r'\b', text, re.IGNORECASE):
            start, end = match.span()
            
            # Determine entity type based on skill
            entity_type = determine_entity_type(skill)  # You'll need to implement this function
            
            annotations.append({
                "start": start,
                "end": end,
                "label": entity_type
            })
    
    # Add to training data
    training_data.append({
        "text": text,
        "annotations": annotations
    })

# Save as JSON file for training
with open('ner_training_data.json', 'w') as f:
    json.dump(training_data, f)

In [ ]:
# Set seed for reproducibility
random.seed(42)

# Shuffle data
random.shuffle(training_data)

# Split into training (80%) and validation (20%)
split_point = int(len(training_data) * 0.8)
train_data = training_data[:split_point]
val_data = training_data[split_point:]

# Save split data
with open('ner_train.json', 'w') as f:
    json.dump(train_data, f)

with open('ner_val.json', 'w') as f:
    json.dump(val_data, f)

In [ ]:


# Load a model pre-trained for NER
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER", num_labels=len(your_label_list))

# Fix your dataset conversion function
def convert_to_hf_dataset(data_list):
    texts = [item["text"] for item in data_list]
    dataset_dict = {"text": texts}
    tags = []
    
    for item in data_list:
        text = item["text"]
        annotations = item["annotations"]
        
        # Get encoding with offset mapping
        encoding = tokenizer(
            text,
            return_offsets_mapping=True,
            add_special_tokens=False,
            truncation=True,
            max_length=512
        )
        
        offset_mapping = encoding["offset_mapping"]
        tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
        token_tags = ["O"] * len(tokens)
        
        # Map character-level annotations to token-level
        for anno in annotations:
            start, end = anno["start"], anno["end"]
            label = anno["label"]
            
            # Find which tokens correspond to this span
            token_start, token_end = None, None
            
            for i, (start_char, end_char) in enumerate(offset_mapping):
                if start_char <= start < end_char:
                    token_start = i
                if start_char < end <= end_char:
                    token_end = i
            
            if token_start is not None and token_end is not None:
                # B- prefix for the first token of an entity
                token_tags[token_start] = f"B-{label}"
                # I- prefix for continuation tokens
                for i in range(token_start + 1, token_end + 1):
                    token_tags[i] = f"I-{label}"
        
        tags.append(token_tags)
    
    dataset_dict["tags"] = tags
    return Dataset.from_dict(dataset_dict)

# Create train and validation datasets
train_dataset = convert_to_hf_dataset(train_data)
val_dataset = convert_to_hf_dataset(val_data)

Token indices sequence length is longer than the specified maximum sequence length for this model (817 > 512). Running this sequence through the model will result in indexing errors


AttributeError: 'RobertaTokenizerFast' object has no attribute 'get_offset_mapping'

In [ ]:
ground_truth_df.columns

Index(['raw_text', 'extracted_role', 'clean_text', 'skills', 'confidence',
       'GT', 'processed'],
      dtype='object')

In [ ]:
def prepare_ner_data(df):
    data_list = []
    
    for idx, row in df.iterrows():
        text = row['raw_text']  # or clean_text if preferred
        skills_list = row['skills']  # List of raw skills with no position info
        
        # Initialize annotations list
        annotations = []
        
        # For each skill, find it in the text and determine its entity type
        for skill in skills_list:
            # Escape special regex characters if present in the skill
            skill_pattern = re.escape(skill)
            
            # Find all occurrences of the skill in the text
            for match in re.finditer(r'\b' + skill_pattern + r'\b', text, re.IGNORECASE):
                start, end = match.span()
                skill_type = determine_entity_type(skill)
                
                annotations.append({
                    "start": start,
                    "end": end,
                    "label": skill_type
                })
        
        data_list.append({
            "text": text,
            "annotations": annotations
        })
    
    return data_list

In [ ]:
# Create train/val split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(ground_truth_df, test_size=0.2, random_state=42)

# Create data with annotations
train_data = prepare_ner_data(train_df)
val_data = prepare_ner_data(val_df)

# Create label list from keyword_mappings
labels = ["O"]  # Start with "O" for non-entity tokens
for skill_type in keyword_mappings.keys():
    labels.append(f"B-{skill_type}")
    labels.append(f"I-{skill_type}")

# Add MISC for anything that doesn't match
labels.append("B-MISC")
labels.append("I-MISC")

# Create label mappings
label_to_id = {label: i for i, label in enumerate(labels)}
id_to_label = {i: label for i, label in enumerate(labels)}

# Load tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

# Convert to HF datasets
def convert_to_hf_dataset(data_list):
    texts = [item["text"] for item in data_list]
    dataset_dict = {"text": texts, "annotations": [item["annotations"] for item in data_list]}
    return Dataset.from_dict(dataset_dict)

train_dataset = convert_to_hf_dataset(train_data)
val_dataset = convert_to_hf_dataset(val_data)

# Tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["text"], 
        truncation=True, 
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True
    )
    
    labels = []
    for i, annotations in enumerate(examples["annotations"]):
        label_ids = [label_to_id["O"]] * len(tokenized_inputs["input_ids"][i])
        offset_mapping = tokenized_inputs["offset_mapping"][i]
        
        for annotation in annotations:
            start_char = annotation["start"]
            end_char = annotation["end"]
            label = annotation["label"]
            
            # Find tokens corresponding to this entity
            token_start_index = None
            token_end_index = None
            
            for j, (start, end) in enumerate(offset_mapping):
                if start == end:  # Skip special tokens
                    continue
                    
                if token_start_index is None and start <= start_char < end:
                    token_start_index = j
                    
                if end > 0 and start < end_char <= end:
                    token_end_index = j
            
            if token_start_index is not None and token_end_index is not None:
                # Assign B- label to first token
                label_ids[token_start_index] = label_to_id[f"B-{label}"]
                
                # Assign I- label to remaining tokens
                for j in range(token_start_index + 1, token_end_index + 1):
                    label_ids[j] = label_to_id[f"I-{label}"]
        
        # Set special tokens to -100
        for j, input_id in enumerate(tokenized_inputs["input_ids"][i]):
            if j == 0 or j == len(label_ids) - 1 or input_id == tokenizer.pad_token_id:
                label_ids[j] = -100
        
        labels.append(label_ids)
    
    # Remove offset_mapping from the features
    tokenized_inputs.pop("offset_mapping")
    tokenized_inputs["labels"] = labels
    
    return tokenized_inputs

# Apply tokenization to datasets
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_and_align_labels, batched=True)

# Load model
model = AutoModelForTokenClassification.from_pretrained(
    "dslim/bert-base-NER", 
    num_labels=len(labels),
    id2label=id_to_label,
    label2id=label_to_id
)

# Set up training arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)

# Define metrics computation
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    predictions, labels = pred
    predictions = predictions.argmax(axis=2)
    
    # Only use valid predictions (not -100)
    true_predictions = [
        [id_to_label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id_to_label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    # Flatten lists
    flat_predictions = [p for sublist in true_predictions for p in sublist]
    flat_labels = [l for sublist in true_labels for l in sublist]
    
    # Calculate metrics
    results = precision_recall_fscore_support(
        flat_labels,
        flat_predictions,
        average='weighted'
    )
    
    return {
        "precision": results[0],
        "recall": results[1],
        "f1": results[2]
    }

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./ner_skills_model")
tokenizer.save_pretrained("./ner_skills_model")

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

TypeError: 'float' object is not iterable